In [ ]:
%matplotlib inline

# *Hubble y los orígenes de DESI*

¡El año 1929 nos trajo los Oscar, la primera radio de coche y la inesperada observación de Edwin Hubble de que todas las galaxias se están alejando de nosotros!

![title](images/edwin-hubble.jpg)

Echemos un vistazo rápido a algunas de las galaxias que estaba viendo, Triángulo y la Gran Nube de Magallanes.

![title](images/Triangulum.jpg)
![title](images/LMC.jpg)

En total, Edwin estudió la distancia de nosotros a 24 galaxias, y sus 'desplazamientos al rojo' observados. ¿Qué significa eso?

Tal vez ya sepas que los niveles de energía del hidrógeno están __cuantificados__, con electrones que habitan una serie de capas con __energías__ discretas. Cuando un electrón transita de un nivel de mayor energía a otro de menor, se emite luz con una longitud de onda  dada específicamente por la fórmula de "Rydberg":

$$ \frac{1}{\lambda_{\rm vac}} = 1.096 \times 10^{7} \left ( \frac{1}{n^2} - \frac{1}{m^2} \right ) $$

donde $n$ y $m$ (cualquiera de $[0, 1, 2, ... \infty]$) etiquetan los dos niveles de energía.

In [ ]:
# Primero, importemos algunos paquetes útiles :
import astropy
import pylab           as     pl
import pandas          as     pd
import numpy           as     np

from   matplotlib      import pyplot as plt
from   scipy           import stats
from   IPython.display import Image
from   tools.wave2rgb  import wavelength_to_rgb

In [ ]:
def Rydberg(n, m):
    # Longitud de onda [nanometros]
    result = 1.096e-2 * (1. / n / n - 1. / m / m)
    return 1. / result

Veamos qué longitudes de onda de la luz que puede emitir el hidrógeno:

In [ ]:
waves = []

print('n \t m \t Wavelength [nm]')

for n in np.arange(1, 10, 1):
    for m in np.arange(n+1, 10, 1):
        wave = Rydberg(n, m)        
        waves.append(wave)
        
        print('{:d} \t {:d} \t {:.3f}'.format(n, m, wave))

Ahora grafiquemos las longitudes de onda y veamos el color de estas líneas. Si tuviéramos que mirar un átomo de hidrógeno emisor, veríamos esto:

In [ ]:
for wave in waves:
  # color = [r, g, b]
  color = wavelength_to_rgb(wave)       
  pl.axvline(x=wave, c=color)

pl.xlabel('Vacuum wavelength [nanometers]')
pl.xlim(380., 780.)

Si existe hidrógeno en una galaxia que se está moviendo, vemos que las líneas se desplazan por efecto Doppler. Digamos que la galaxia se mueve al 1% de la velocidad de la luz.

In [ ]:
def redshift(v):
    # v [velocidad de la Luz].
    result = (1. + v) / (1. - v) 
    result = np.sqrt(result) - 1.
    
    return  result

In [ ]:
zz = redshift(0.01) 

for restwave in waves:
  obswave = (1. + zz) * restwave     

  color   = wavelength_to_rgb(restwave)       
  pl.axvline(x=restwave, c=color, alpha=0.25)

  color   = wavelength_to_rgb(obswave)       
  pl.axvline(x=obswave, c=color)

pl.xlabel('Vacuum wavelength [nanometers]')
pl.xlim(380., 780.)

Aquí puedes ver la línea original (tenue) y la línea desplazada debido a que la galaxia con el hidrógeno emisor se está moviendo. [Aquí](https://es.wikipedia.org/wiki/Efecto_Doppler) encontrarás todos los detalles.

Hubble conocía las líneas del hidrógeno y muchos otros elementos. Al invertir lo anterior, pudo calcular la velocidad de muchas galaxias. Descubrió qué tan lejos estaban (usando la información de qué tan brillantes eran algunas [estrellas especiales en la galaxia](https://es.wikipedia.org/wiki/Estrella_variable_Cefeida) y qué tan rápido se estaban moviendo (a partir de su desplazamiento al rojo, ver arriba):

In [ ]:
dat = pd.read_csv('dat/hubble.dat', sep='\s+', comment='#', names=['Galaxy name', 'Distance [Mpc]', 'Velocity [km/s]'])
dat

Vamos a graficarlas.

In [ ]:
fig = plt.figure(figsize=(10, 7.5))
ax  = fig.add_subplot(1, 1, 1)
plt.close()

In [ ]:
label_style = {'fontname': 'Georgia', 'fontsize': 16} 

In [ ]:
ax.plot(dat['Distance [Mpc]'], dat['Velocity [km/s]'], '-', c='k', marker='*', lw=0)

ax.set_xlabel('Distancia desde Nosotros [Megaparsecs]', **label_style)
ax.set_ylabel('Velocida de Recesión [km/s]', **label_style)

plt.tight_layout()

In [ ]:
fig

Edwin vio una tendencia clara, pero las mediciones parecían bastante ruidosas. Intentemos hacer nuestra mejor suposición sobre la verdadera relación entre los dos. Construiremos una relación lineal (regresión) usando el paquete scipy stats:

In [ ]:
slope, intercept, r_value, p_value, std_err = stats.linregress(dat['Distance [Mpc]'],dat['Velocity [km/s]'])

In [ ]:
print('La pendiente de esta tendencia (slope) es conocida como la constante de Hubble:  {:.3f} [km/s/Mpc]'.format(slope))

¿Cómo se ve esto?.

In [ ]:
distances  = np.linspace(-0.5, 2.5, 10)
velocities = slope * distances

ax.plot(distances, velocities, lw=0.25, c='k')
ax.set_xlim(0.0, 2.5)

In [ ]:
fig

¡Parece un ajuste bastante bueno!

Ahora es tu turno, ¿puedes hacer una buena estimación del error en esta medición de la constante del Hubble? ¿Con qué precisión podemos predecir la recesión de una galaxia a una distancia determinada, es decir, qué tan rápido o lento podría moverse?

Entonces, en conclusión, ¡es probable que todas las galaxias se alejen de nosotros! Descubrimos que esto es cierto para todas las galaxias: no estamos en el centro ni somos especiales de ninguna manera. Cada galaxia se aleja de las demás. El hecho de que el Universo se estuviera expandiendo fue un shock para muchos en 1929, pero les esperaba una sorpresa aún mayor.

# *Energía oscura*

En 1998, el mundo cambiaría para siempre. Larry Page y Sergey Brin fundaron Google, el nodo American Unity y el módulo ruso Zarya se unirían para formar la [Estación Espacial Internacional](https://es.wikipedia.org/wiki/Estación_Espacial_Internacional ) y Saul Perlmutter (del laboratorio Lawrence Berkeley), Brian Schmidt y Adam Reiss, confirmaron irrefutablemente la existencia de la _Energía Oscura_ (_Dark Energy_). Aquí está Saul impresionando a algunos jóvenes investigadores de Berkeley con estos resultados en ese momento:

![title](images/perlmutter.png)

Entonces, ¿qué estaban mirando todos? Analicemos los datos.

In [ ]:
dat     = pd.read_csv('dat/perlmutter.txt', names=['z', 'Effective magnitude'], comment='#', sep='\s+')
toprint = dat[:10]
toprint

Un gráfico mostraría esto mucho más claramente:

In [ ]:
pl.plot(dat['z'], dat['Effective magnitude'], marker='.', lw=0.0)

pl.xlabel('z')
pl.ylabel('Effective magnitude')

Saul tuvo buenas razones para creer (en realidad, primero tuvo que modificarlas un poco) que todas las [supernovas de tipo Ia](https://es.wikipedia.org/wiki/Supernova_de_tipo_Ia) que se muestran aquí eran igualmente brillantes intrínsecamente, pero aquellas con alto desplazamiento al rojo parecían relativamente débiles en comparación con aquellas con bajo desplazamiento al rojo, ya que simplemente estaban más lejos. Esto explica la tendencia mostrada, dado que la 'magnitud efectiva' es la forma rara en la que los astrónomos suelen expresar cuán brillante parece algo.

Lo útil de esta medición es que la distancia a la que se encuentra una supernova o galaxia para un desplazamiento al rojo determinado depende de algunos parámetros, uno de los cuales es la cantidad de energía oscura que podría haber en el Universo. Casi todos esperaban que estos datos demostraran que no había _ninguna_ _Energía Oscura_ cuando Saul lo hizo, pero algunos adivinaron lo contrario.

Cuando Hubble descubrió la expansión, una consecuencia natural fue que la cantidad de energía (masa en reposo) contenida en un metro cúbico se diluiría con el tiempo. La _Energía Oscura_ sería especial, ya que la cantidad de energía por metro cúbico sería constante con el tiempo y sugeriría que algunos efectos espeluznantes de la [mecánica cuántica](https://es.wikipedia.org/wiki/Mecánica_cuántica) estarían causando que las galaxias se separen.

Así que, usemos los datos de Saul para averiguar cuánta energía oscura hay en el universo. Primero, necesitamos un modelo para la distancia (luminosidad) de una supernova con un desplazamiento al rojo dado, dada cierta cantidad de Energía Oscura. Usamos $\Omega_\Lambda$ para denotar la _fracción_ de toda la materia que se comporta como Energía Oscura.

In [ ]:
from astropy.cosmology import FlatLambdaCDM

def lumdist(z, olambda):
  cosmo = FlatLambdaCDM(H0=70, Om0=1. - olambda, Tcmb0=2.725)
    
  return  cosmo.luminosity_distance(z)   

Luego, necesitamos convertir esta distancia en la forma en que los astrónomos miden el brillo:

In [ ]:
def effmag(z, olambda, MB):
  DL = lumdist(z, olambda)   

  return MB + 5. * np.log10(DL.value)    

In [ ]:
zs = np.arange(0.01, 0.85, 0.01)

pl.plot(dat['z'], dat['Effective magnitude'], marker='.', lw=0.0)

pl.plot(zs, effmag(zs, 0.0, 6.), c='k', label='No Dark Energy', alpha=0.5)
pl.plot(zs, effmag(zs, 0.5, 6.), c='k', label='Dark Energy!')

pl.xlabel('z')
pl.ylabel('Magnitud Efectiva')

pl.legend(loc=4, frameon=False)

Incluso a simple vista, los datos parecen preferir algo de energía oscura. Pero no hay una gran cantidad de ésta. Averigüemos qué prefieren exactamente los datos. Para hacer esto, asumimos que minimizar la distancia entre cada punto y la línea es la mejor medida de qué tan bien se ajusta la teoría a los datos (consulta este [link](https://es.wikipedia.org/wiki/M%C3%ADnimos_cuadrados)). Además de la fracción de energía oscura, tampoco sabemos qué tan brillante es intrínsecamente cada supernova, por lo que ajustaremos ambas simultáneamente.

In [ ]:
from scipy.optimize import minimize

In [ ]:
def chi2(x):
     olambda = x[0]
     MB      = x[1] 
    
     model   = effmag(dat['z'], olambda, MB) 
     
     return  np.sum((dat['Effective magnitude'] - model)**2.) 

In [ ]:
res = minimize(chi2, x0=[0.5, 5.0], options={'disp': True})

In [ ]:
res.x

In [ ]:
zs = np.arange(0.01, 0.85, 0.01)

pl.plot(dat['z'], dat['Effective magnitude'], marker='.', lw=0.0)

pl.plot(zs, effmag(zs, 0.0, 6.), c='k',   label='No Dark Energy', alpha=0.5)
pl.plot(zs, effmag(zs, 0.5, 6.), c='k',   label='50% Dark Energy!')
pl.plot(zs, effmag(zs, 0.75, 6.), c='c', label='75% Dark Energy!')

pl.xlabel('z')
pl.ylabel('Magnitud efectiva')

pl.legend(loc=4, frameon=False)

¡Así que hay algo así como 75% de energía oscura en el Universo! Siendo las primeras personas en realizar esta medición, Saul, junto con Brian Schmidt y Adam Reiss, recibirían el Premio Nobel 2011 por su trabajo.

![title](images/perlmutter_nobel.jpg)

Puedes encontrar todos los detalles [aquí](https://arxiv.org/pdf/astro-ph/9812133.pdf). Advertencia, esto es para los profesionales, así que no te preocupes si no comprendes mucho.

La principal motivación para DESI es repetir mediciones similares de desplazamientos al rojo de galaxias distantes, con mucha más precisión y aprender mucho más sobre esta espeluznante Energía Oscura.